In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install scann

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 14.2 MB/s 
     |████████████████████████████████| 511.8 MB 9.5 kB/s 
     |████████████████████████████████| 438 kB 66.8 MB/s 
     |████████████████████████████████| 5.8 MB 38.7 MB/s 
     |████████████████████████████████| 1.6 MB 63.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unin

In [ ]:
import numpy as np
import h5py
import os
import requests
import tempfile
import time
import pickle

import scann

In [ ]:
data_dir = '/content/drive/MyDrive/MedImageCap/openi_image_embed_new/'

openi_feats = np.load(data_dir + 'openi_img_feats.npy',allow_pickle=True)

# openi_feats = openi_feats / np.linalg.norm(openi_feats, axis=1)[:, np.newaxis]

train_idx = np.load(data_dir + 'openi_train_img_idx.npy',allow_pickle=True)
val_idx = np.load(data_dir + 'openi_val_img_idx.npy',allow_pickle=True)
test_idx = np.load(data_dir + 'openi_test_img_idx.npy',allow_pickle=True)


train_feats = openi_feats[train_idx]
val_feats = openi_feats[val_idx]
test_feats = openi_feats[test_idx]


with open( data_dir + 'openi_img_ids', 'rb') as f:
  openi_img_ids = pickle.load(f)


with open( data_dir + 'openi_train_img_ids', 'rb') as f:
  train_img_ids = pickle.load(f)

with open( data_dir + 'openi_val_img_ids', 'rb') as f:
  val_img_ids = pickle.load(f)

with open( data_dir + 'openi_test_img_ids', 'rb') as f:
  test_img_ids = pickle.load(f)

print(train_feats.shape)
print(val_feats.shape)
print(test_feats.shape)

(2566, 1024)
(366, 1024)
(733, 1024)


In [ ]:
train_img_ids[0]

'CXR3403_IM-1647-1001'

In [ ]:
val_img_ids[0]

'CXR3518_IM-1717-1001'

In [ ]:
print(np.min(train_feats[0]))
print(np.max(train_feats[0]))

-1.4169394
1.1799977


In [ ]:
normalized_dataset = train_feats / np.linalg.norm(train_feats, axis=1)[:, np.newaxis]

# normalized_train = train_feats

In [ ]:
normalized_dataset.shape

(2566, 1024)

In [ ]:
print(np.min(normalized_dataset[0]))
print(np.max(normalized_dataset[0]))

-0.10882849
0.090630114


In [ ]:
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 6, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=10000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [ ]:
# increasing the leaves to search increases recall at the cost of speed
start = time.time()
train_neighbors, train_distances = searcher.search_batched(train_feats, leaves_to_search=200)
end = time.time()

In [ ]:
train_neighbors.shape

(2566, 6)

In [ ]:
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 5, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=10000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [ ]:
start = time.time()
val_neighbors, val_distances = searcher.search_batched(val_feats, leaves_to_search=200)
end = time.time()

In [ ]:
val_neighbors.shape

(366, 5)

In [ ]:
start = time.time()
test_neighbors, test_distances = searcher.search_batched(test_feats, leaves_to_search=150)
end = time.time()

In [ ]:
test_neighbors.shape

(733, 5)

In [ ]:
def get_image_edge_index(neighbours,distances,train_img_ids,val_img_ids=None,data='train'):
    
    n,m = neighbours.shape
    edges = []
    edge_weights = []
    for i in range(n):
        for j in range(m):
            if(data == 'train'):
                if(i != neighbours[i][j]):
                    edges.append([train_img_ids[i],train_img_ids[neighbours[i][j]]])
                    edge_weights.append(distances[i][j])
            else:
                edges.append([val_img_ids[i],train_img_ids[neighbours[i][j]]])
                edge_weights.append(distances[i][j])
                
    return np.array(edges),np.array(edge_weights)

In [ ]:
train_img_edges,train_img_edge_weights = get_image_edge_index(train_neighbors,train_distances,train_img_ids)
val_img_edges,val_img_edge_weights = get_image_edge_index(val_neighbors,val_distances,train_img_ids,val_img_ids,data='val')
test_img_edges,test_img_edge_weights = get_image_edge_index(test_neighbors,test_distances,train_img_ids,test_img_ids,data='test')

In [ ]:
np.save(data_dir + 'openi_train_img_neighbours',train_neighbors)
np.save(data_dir + 'openi_val_img_neighbours',val_neighbors)
np.save(data_dir + 'openi_test_img_neighbours',test_neighbors)


np.save(data_dir + 'openi_train_img_edges',train_img_edges)
np.save(data_dir + 'openi_val_img_edges',val_img_edges)
np.save(data_dir + 'openi_test_img_edges',test_img_edges)

np.save(data_dir + 'openi_train_img_edge_weights',train_img_edge_weights)
np.save(data_dir + 'openi_val_img_edge_weights',val_img_edge_weights)
np.save(data_dir + 'openi_test_img_edge_weights',test_img_edge_weights)

In [ ]:
train_distances[0]

array([13.019932, 12.740858, 12.710445, 12.63843 , 12.581247, 12.551917],
      dtype=float32)

In [ ]:
train_img_edges[:10]

array([['CXR3403_IM-1647-1001', 'CXR1951_IM-0619-1001'],
       ['CXR3403_IM-1647-1001', 'CXR3117_IM-1465-1001'],
       ['CXR3403_IM-1647-1001', 'CXR3257_IM-1544-1001'],
       ['CXR3403_IM-1647-1001', 'CXR1502_IM-0328-1001'],
       ['CXR3403_IM-1647-1001', 'CXR3421_IM-1656-1001'],
       ['CXR810_IM-2343-1001', 'CXR3911_IM-1987-1001'],
       ['CXR810_IM-2343-1001', 'CXR3555_IM-1741-2001'],
       ['CXR810_IM-2343-1001', 'CXR2609_IM-1099-3001'],
       ['CXR810_IM-2343-1001', 'CXR2136_IM-0758-3001'],
       ['CXR810_IM-2343-1001', 'CXR2020_IM-0668-1001']], dtype='<U25')

In [ ]:
train_img_edges[:10]

array([['CXR3403_IM-1647-1001', 'CXR1951_IM-0619-1001'],
       ['CXR3403_IM-1647-1001', 'CXR3117_IM-1465-1001'],
       ['CXR3403_IM-1647-1001', 'CXR3257_IM-1544-1001'],
       ['CXR3403_IM-1647-1001', 'CXR1502_IM-0328-1001'],
       ['CXR3403_IM-1647-1001', 'CXR3421_IM-1656-1001'],
       ['CXR810_IM-2343-1001', 'CXR3911_IM-1987-1001'],
       ['CXR810_IM-2343-1001', 'CXR3555_IM-1741-2001'],
       ['CXR810_IM-2343-1001', 'CXR2609_IM-1099-3001'],
       ['CXR810_IM-2343-1001', 'CXR2136_IM-0758-3001'],
       ['CXR810_IM-2343-1001', 'CXR2020_IM-0668-1001']], dtype='<U25')

In [ ]:
val_img_edges[:10]

array([['CXR3518_IM-1717-1001', 'CXR2170_IM-0786-1001'],
       ['CXR3518_IM-1717-1001', 'CXR3112_IM-1461-1001'],
       ['CXR3518_IM-1717-1001', 'CXR376_IM-1883-1001'],
       ['CXR3518_IM-1717-1001', 'CXR262_IM-1109-1001'],
       ['CXR3518_IM-1717-1001', 'CXR2119_IM-0746-1001'],
       ['CXR2553_IM-1059-1001', 'CXR1471_IM-0304-2001'],
       ['CXR2553_IM-1059-1001', 'CXR2018_IM-0665-1001'],
       ['CXR2553_IM-1059-1001', 'CXR2163_IM-0779-1001'],
       ['CXR2553_IM-1059-1001', 'CXR82_IM-2350-3001'],
       ['CXR2553_IM-1059-1001', 'CXR2680_IM-1154-1001']], dtype='<U25')

In [ ]:
val_img_edges[:50]

array([['CXR3518_IM-1717-1001', 'CXR2170_IM-0786-1001'],
       ['CXR3518_IM-1717-1001', 'CXR3112_IM-1461-1001'],
       ['CXR3518_IM-1717-1001', 'CXR376_IM-1883-1001'],
       ['CXR3518_IM-1717-1001', 'CXR262_IM-1109-1001'],
       ['CXR3518_IM-1717-1001', 'CXR2119_IM-0746-1001'],
       ['CXR2553_IM-1059-1001', 'CXR1471_IM-0304-2001'],
       ['CXR2553_IM-1059-1001', 'CXR2018_IM-0665-1001'],
       ['CXR2553_IM-1059-1001', 'CXR2163_IM-0779-1001'],
       ['CXR2553_IM-1059-1001', 'CXR82_IM-2350-3001'],
       ['CXR2553_IM-1059-1001', 'CXR2680_IM-1154-1001'],
       ['CXR3156_IM-1486-1001', 'CXR1356_IM-0231-1001'],
       ['CXR3156_IM-1486-1001', 'CXR3770_IM-1890-1001'],
       ['CXR3156_IM-1486-1001', 'CXR1384_IM-0246-1001'],
       ['CXR3156_IM-1486-1001', 'CXR3663_IM-1822-1001'],
       ['CXR3156_IM-1486-1001', 'CXR2467_IM-0997-1001'],
       ['CXR3883_IM-1971-1001', 'CXR2244_IM-0841-2002'],
       ['CXR3883_IM-1971-1001', 'CXR3173_IM-1495-2002'],
       ['CXR3883_IM-1971-1001', 'CX

In [ ]:
train_distances[0]

array([13.019932, 12.740858, 12.710445, 12.63843 , 12.581247, 12.551917],
      dtype=float32)

In [ ]:
train_neighbors[:10]

array([[   0, 1789, 1552, 1985, 1339,  895],
       [   1, 1677,  911, 2141, 1383, 1570],
       [   2, 1340, 1827,  880, 1375,  545],
       [   3, 1835, 1804,  634, 1639, 1637],
       [   4,  918, 2090, 1408,  817, 2095],
       [   5,  613,  688, 1754, 1594, 1899],
       [   6, 2475,  422,  457,  369, 2378],
       [   7, 1277,  180, 1777, 2389,  885],
       [   8,  459, 2293,  903,  223, 1171],
       [   9,  885, 1500, 1985,  650, 1572]], dtype=uint32)

In [ ]:
train_neighbors.shape

(2566, 6)

In [ ]:
val_distances[0]

array([14.255459, 14.218697, 14.211312, 14.201454, 14.18778 ],
      dtype=float32)

In [ ]:
train_neighbors[120]

array([ 120, 1120, 1760, 2361, 2195, 1265], dtype=uint32)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
x = val_feats[0].reshape(1,-1)
x.shape

(1, 1024)

In [ ]:
y = train_feats

In [ ]:
sim = cosine_similarity(x,y)

In [ ]:
sim.shape

(1, 2566)

In [ ]:
np.argpartition(sim.reshape(-1), -10)[-10:]

array([ 924,  411, 1086, 2481, 1201, 2361, 1908,  194, 1466,  764])

In [ ]:
val_neighbors[0]

array([1086, 2481,  764,  194, 1466], dtype=uint32)